In [190]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [191]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder , MinMaxScaler
from sklearn.pipeline import Pipeline , make_pipeline
from sklearn.feature_selection import SelectKBest , chi2
from sklearn.tree import DecisionTreeClassifier

In [192]:
df = pd.read_csv('/content/train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [193]:
df.drop(columns = ["PassengerId","Name","Ticket","Cabin"] , inplace = True)

In [194]:
X_train , X_test , y_train , y_test =  train_test_split(df.drop(columns = ["Survived"] , axis = 1) ,
                                                        df["Survived"] ,
                                                        test_size = 0.2 ,
                                                        random_state = 42)

In [195]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [226]:
### Simple Imputer Column Transformer

trf1 = ColumnTransformer(
    transformers= [
        ("impute_age" , SimpleImputer() , [2]),
        ("impute_embarked" , SimpleImputer(strategy = "most_frequent") , [6])
    ],
    remainder = "passthrough"
)

In [227]:
## One Hot Encoding Column transformer

trf2 = ColumnTransformer(
    transformers = [
        ("ohe_sex_embarked" , OneHotEncoder(sparse_output = False  , dtype = np.int32) , [1,3])
    ],
    remainder = "passthrough"
)

In [228]:
### Scaling

trf3 = ColumnTransformer(
    transformers = [
        ("scale" , MinMaxScaler() , slice(0,8))
    ],
    remainder = "passthrough"
)

In [229]:
## Feature Scaling
trf4 = SelectKBest(score_func= chi2 , k = 8)

In [230]:
## train the model
tfr5 = DecisionTreeClassifier()

# **Create Pipeline**

In [231]:
pipe = Pipeline([
    ("trf1" , trf1),
    ("trf2" , trf2),
    ("trf3" , trf3),
    ("trf4" , trf4),
    ("tfr5" , tfr5)
])

***Pipeline Vs make_pipeline***
We should use make_pipeline instead of pipeline because it is simple

In [202]:
pipe1 = make_pipeline(trf1 , trf2 , trf3 , trf4 , tfr5)

In [232]:
pipe.fit(X_train , y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(dtype=<class 'numpy.int32'>,
                                                                sparse_output=False),
                                                  [1, 3])])),
                ('trf3',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 8, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x7a9a073260e0>)),
                ('tfr5', DecisionTreeClassifier())])

# **Explore the Pipe**

In [233]:
pipe.named_steps

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'trf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(dtype=<class 'numpy.int32'>,
                                                sparse_output=False),
                                  [1, 3])]),
 'trf3': ColumnTransformer(remainder='passthrough',
                   transformers=[('scale', MinMaxScaler(), slice(0, 8, None))]),
 'trf4': SelectKBest(k=8, score_func=<function chi2 at 0x7a9a073260e0>),
 'tfr5': DecisionTreeClassifier()}

In [246]:
pipe.named_steps["trf1"].transformers_[0][1].statistics_

array([29.49884615])

In [253]:
pipe.named_steps["trf1"].transformers_[1][1].statistics_

array(['S'], dtype=object)

In [254]:
# Predict
y_pred = pipe.predict(X_test)

In [255]:
y_pred

array([0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 1])

In [256]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test , y_pred)

0.7877094972067039

# **Cross Validation Using Pipeline**

In [257]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe , X_train , y_train ,cv = 5 , scoring = "accuracy").mean()

0.7908598443809712

# **Grid Search CV Using Pipeline**

In [261]:
params = {
    "tfr5__max_depth" : [1,2,3,4,5,None]
}

from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe , params , cv = 5 , scoring = "accuracy")
grid.fit(X_train , y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('trf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(dtype=<class 'numpy.int32'>,
                                                                                       sparse_output=False),
                                                                         [1,
                                                                          3])])),
                                       ('trf3',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 8, None))])),
                                       ('trf4',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x7a9a073260e0>)),
                                       ('tfr5', DecisionTreeClassifier())]),
             param_grid={'tfr5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [263]:
grid.best_score_

0.8033093666896484

In [264]:
grid.best_params_

{'tfr5__max_depth': 3}

# **Exporting Pipeline**

In [265]:
import pickle
pickle.dump(pipe , open("pipe.pkl","wb"))

# **Importing Model**

In [266]:
model = pickle.load(open("pipe.pkl","rb"))

In [267]:
test_input = np.array([[2,"male", 31.0 , 0 , 0 , 10.5 , "S"]] , dtype="object").reshape(1,7)
test_input

array([[2, 'male', 31.0, 0, 0, 10.5, 'S']], dtype=object)

In [270]:
classes = ["Not Survived" ,"Survived"]
classes[model.predict(test_input)[0]]

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


'Not Survived'